In [1]:
import os
import numpy as np
import pandas as pd
import scanpy as sc

import muon as mu

import warnings
warnings.filterwarnings('ignore')

/home/dbdimitrov/miniconda3/envs/spiana/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

import liana as li
import plotnine as p9

In [3]:
import decoupler as dc
from functools import reduce

In [4]:
path = os.path.join('..','..', 'data', 'sma')

## Load Prior Knowledge

### Protein-metabolite interactions

In [5]:
# Load metabolite receptor interactions
resource = pd.read_csv(os.path.join("resources", 'MR_brain_20230808.csv'))
resource['receptor'] = resource['receptor'].str.title()
resource.head()

,Unnamed: 0,ligand,receptor
0,1,HMDB0000112,Gabrb2
1,3,HMDB0000216,Adra2B
2,4,HMDB0000068,Adra2B
3,20,HMDB0000148,Grm8
4,49,HMDB0000112,Gabra5


In [6]:
receptors = resource['receptor'].unique()

### PPI* PKN

In [7]:
input_pkn = pd.read_csv(os.path.join("resources", "cosmos_network.csv"))
input_pkn = input_pkn.rename(columns={"interaction":'mor'})

In [8]:
# remove weird stuff from metab, keep only HMDB
input_pkn.loc[input_pkn['source'].str.contains("HMDB"), 'source'] = input_pkn[input_pkn['source'].str.contains("HMDB")]['source'].str.split("__").str[1]
# remove _c from source
input_pkn['source'] = input_pkn['source'].str.replace("_c", "")

In [9]:
# if it does not contain HMDB then to title
input_pkn.loc[~input_pkn['source'].str.contains("HMDB"), 'source'] = input_pkn[~input_pkn['source'].str.contains("HMDB")]['source'].str.title()
input_pkn.loc[~input_pkn['target'].str.contains("HMDB"), 'target'] = input_pkn[~input_pkn['target'].str.contains("HMDB")]['target'].str.title()

## Load Slides

In [10]:
slides = pd.read_csv(os.path.join(path, "sma_meta.csv"), index_col=0).index
exp = "V11L12-109" # V11T17-102 V11L12-109 V11T16-085

In [11]:
hvg = []
hvm = []
hvc = []
mdatas = {}

for slide in slides:    
    slide_path = os.path.join(path, exp, f"{exp}_{slide}", "output_data")
    mdatas[slide] = mu.read_h5mu(os.path.join(slide_path, "sma.h5mu"))
    msi = mdatas[slide].mod["msi"]
    rna = mdatas[slide].mod["rna"]
    ct = mdatas[slide].mod["deconv"]
    
    # Keep most variable metabolites
    sc.pp.highly_variable_genes(msi, flavor='cell_ranger', n_top_genes=250)
    
    # Remove predictors with no variation 
    sc.pp.highly_variable_genes(rna, flavor='cell_ranger', n_top_genes=12500)
    ct.var['cv'] = ct.X.var(axis=0) / ct.X.mean(axis=0)
    ct.var['highly_variable'] = ct.var['cv'] > np.percentile(ct.var['cv'], 20)
    
    msk = (msi.var['highly_variable'].values)
    hvm.append(list(msi.var[msk].index))
    hvg.append(list(rna.var[rna.var['highly_variable']].index))
    hvc.append(list(ct.var[ct.var['highly_variable']].index))

In [12]:
# Common highly-variable metabolite peaks
hvm = reduce(np.intersect1d, hvm)
hvg = reduce(np.intersect1d, hvg)
hvc = reduce(np.intersect1d, hvc)

In [13]:
# only keep hvg receptors
receptors = receptors[np.isin(receptors, hvg)]

In [14]:
# check if predictor n are somewhat comparable
print(len(hvm), len(receptors), len(hvc))

83 45 37


## Run MISTy

In [15]:
interaction_list = {}
target_list = {}
bandwidth=1000
cutoff=0.1

for slide in slides:
    mdata = mdatas[slide]
    
    msi = mdata.mod["msi"]
    msi = msi[:, hvm].copy()
    sc.pp.scale(msi, max_value=5)
    msi.X[msi.X < 0] = 0
    
    # distances of metabolties to RNA
    reference = mdata.mod["msi"].obsm["spatial"]
    
    deconv = mdata.mod["deconv"][:, hvc]
    li.ut.spatial_neighbors(deconv, bandwidth=bandwidth, cutoff=cutoff, spatial_key="spatial", reference=reference, set_diag=False, standardize=False)
    
    rna = mdata.mod["rna"]
    li.ut.spatial_neighbors(rna, bandwidth=bandwidth, cutoff=cutoff, spatial_key="spatial", reference=reference, set_diag=False, standardize=False)
    
    # Subset RNA to metabolite receptors only
    rec = rna[:, receptors].copy()

    # MISTy
    mdata.update_obs()
    
    misty = li.mt.MistyData({"intra": msi, "receptor": rec, "ct": deconv}, enforce_obs=False, obs=mdata.obs)
    misty(model="linear", verbose=True, bypass_intra=True, maskby='lesion')
    
    interaction_list[slide] = misty.uns['interactions'].copy()
    target_list[slide] = misty.uns['target_metrics'].copy()

    mdata.write_h5mu(os.path.join('results', f"{slide}_misty.h5mu"))


view intra is not a csr_matrix. Converting to csr_matrix
view ct is not a csr_matrix. Converting to csr_matrix
Now learning: Dopamine masked by lesioned: 100%|██████████| 83/83 [00:18<00:00,  4.44it/s]
view intra is not a csr_matrix. Converting to csr_matrix
view ct is not a csr_matrix. Converting to csr_matrix
Now learning: Dopamine masked by lesioned: 100%|██████████| 83/83 [00:13<00:00,  6.14it/s]
view intra is not a csr_matrix. Converting to csr_matrix
view ct is not a csr_matrix. Converting to csr_matrix
Now learning: Dopamine masked by lesioned: 100%|██████████| 83/83 [00:13<00:00,  6.24it/s]


In [16]:
interactions = pd.concat(interaction_list).reset_index(names=['slide', 'ind']).drop(columns=['ind'])
target_metrics = pd.concat(target_list).reset_index(names=['slide', 'ind']).drop(columns=['ind'])

In [17]:
interactions.to_csv(os.path.join('results', "interactions.csv"), index=False)
target_metrics.fillna(0, inplace=True)
target_metrics.to_csv(os.path.join("results", "target_metrics.csv"), index=False)

## Run Local Scores

In [18]:
for slide in slides:
    mdata = mdatas[slide]
    rna = mdata.mod["rna"]
    msi = mdata.mod["msi"]
    deconv = mdata.mod["deconv"]
    
    # interpolate
    ad = li.ut.interpolate_adata(target=msi, reference=rna, use_raw=False, spatial_key='spatial')
    
    mdata = mu.MuData({'msi': ad, 'rna':rna, 'deconv':deconv}, obsm=rna.obsm, obs=rna.obs, uns=rna.uns)
    li.ut.spatial_neighbors(mdata, bandwidth=bandwidth, cutoff=cutoff, set_diag=True)

    li.mt.bivar(mdata, 
                function_name='cosine',
                x_mod='msi', 
                y_mod='deconv',
                x_use_raw=False, 
                y_use_raw=False,
                verbose=True, 
                mask_negatives=True, 
                add_categories=True,
                n_perms=1000,
                interactions = [('Dopamine', 'MSN1'),
                                ('Dopamine', 'MSN2')],
                x_transform=sc.pp.scale,
                y_transform=sc.pp.scale,
                mod_added='msi_ct'
                )


    li.mt.bivar(mdata, 
                function_name='cosine',
                x_mod='msi', 
                y_mod='rna',
                x_use_raw=False, 
                y_use_raw=False,
                verbose=True, 
                mask_negatives=True, 
                add_categories=True,
                n_perms=1000,
                interactions = [('Dopamine', 'Drd2')],
                x_transform=sc.pp.scale,
                y_transform=sc.pp.scale,
                mod_added='lr'
                )
    
    mdata.update_obs()
    mdata.update_var()
    mdata.write_h5mu(os.path.join('results', f"{slide}_local.h5mu"))

Using provided `interactions`.
Using `.X`!
Using `.X`!
Converting to sparse csr matrix!
Using `.X`!
Converting to sparse csr matrix!


Transforming msi using scale
Transforming deconv using scale


100%|██████████| 1000/1000 [00:00<00:00, 1384.55it/s]
Using provided `interactions`.
Using `.X`!
Using `.X`!


Transforming msi using scale
Transforming rna using scale


Using `.X`!
Converting to sparse csr matrix!
100%|██████████| 1000/1000 [00:00<00:00, 8019.73it/s]
... storing 'x' as categorical
Using provided `interactions`.
Using `.X`!
Using `.X`!
Converting to sparse csr matrix!
Using `.X`!
Converting to sparse csr matrix!


...

Using `.X`!
Converting to sparse csr matrix!
1 features of mat are empty, they will be removed.
100%|██████████| 1000/1000 [00:00<00:00, 6969.24it/s]
... storing 'x' as categorical
Using provided `interactions`.
Using `.X`!
Using `.X`!
Converting to sparse csr matrix!
Using `.X`!
Converting to sparse csr matrix!


Transforming msi using scale
Transforming deconv using scale


100%|██████████| 1000/1000 [00:00<00:00, 1274.84it/s]
Using provided `interactions`.
Using `.X`!
Using `.X`!


Transforming msi using scale
Transforming rna using scale


Using `.X`!
Converting to sparse csr matrix!
100%|██████████| 1000/1000 [00:00<00:00, 7694.46it/s]
... storing 'x' as categorical
